# Mechanotransduction model in a mammalian cell

Here, we implement the model originally presented in [Sun et al. 2016, Biophysical Journal], then expanded by [Scott et al. 2021, PNAS] and [Francis and Rangamani, BioRxiv]. The goal is to describe the nuclear translocation of the mechanosensitive transcriptional regulators YAP/TAZ and MRTF, given some mechanical input from the substrate.

The geometry in this model is divided into 4 domains - two volumes and two surfaces:
- plasma membrane (PM)
- Cytosol (Cyto)
- Nuclear membrane (NM)
- Interior of the nucleus (Nuc)

This model has 15 species:
- 
```

In [1]:
import dolfin as d
import sympy as sym
import numpy as np
import pathlib
import logging
import gmsh # must be imported before pyvista if dolfin is imported first

from smart import config, mesh, model, mesh_tools, visualization
from smart.units import unit
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
    sbmodel_from_locals
)

from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

logger = logging.getLogger("smart")
logger.setLevel(logging.DEBUG)

In [2]:
# Aliases - base units
uM = unit.uM
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = uM * um / sec
vol_unit = uM
surf_unit = molecule / um**2
# stiffness units
kPa = unit.kilopascal

In [3]:
cell_rad = 5.0
nuc_rad = 0.5*cell_rad
cell_mesh, facet_markers, cell_markers = mesh_tools.create_spheres(outerRad = cell_rad, innerRad = nuc_rad,
                                                                   hEdge=0.6, hInnerEdge=0.6)
mesh_folder = pathlib.Path("sphereCell_mesh")
mesh_folder.mkdir(exist_ok=True)
mesh_file = mesh_folder / "sphereCell_mesh.h5"
mesh_tools.write_mesh(cell_mesh, facet_markers, cell_markers, mesh_file)

parent_mesh = mesh.ParentMesh(
    mesh_filename=str(mesh_file),
    mesh_filetype="hdf5",
    name="parent_mesh",
)

 2023-10-23 18:51:04,900 smart.mesh - INFO - HDF5 mesh, "parent_mesh", successfully loaded from file: sphereCell_mesh/sphereCell_mesh.h5! (mesh.py:220) 


In [4]:
Cyto = Compartment("Cyto", 3, um, 1)
PM = Compartment("PM", 2, um, 10)
Nuc = Compartment("Nuc", 3, um, 2)
NM = Compartment("NM", 2, um, 12)
PM.specify_nonadjacency(['NM', 'Nuc'])
NM.specify_nonadjacency(['PM'])

In [5]:
pFAK = Species("pFAK", 0.3, vol_unit, 10.0, D_unit, "Cyto")
RhoA_GDP = Species("RhoA_GDP", 1.0, vol_unit, 1.0, D_unit, "Cyto")
RhoA_GTP = Species("RhoA_GTP", 33.6, surf_unit, 0.3, D_unit, "PM")
ROCK_A = Species("ROCK_A", 0.0, vol_unit, 75.0, D_unit, "Cyto")
mDia_A = Species("mDia_A", 0.0, vol_unit, 1.0, D_unit, "Cyto")
Myo_A = Species("Myo_A", 1.5, vol_unit, 0.8, D_unit, "Cyto")
LIMK_A = Species("LIMK_A", 0.1, vol_unit, 10.0, D_unit, "Cyto")
Cofilin_NP = Species("Cofilin_NP", 1.8, vol_unit, 10.0, D_unit, "Cyto")
FActin = Species("FActin", 17.9, vol_unit, 0.6, D_unit, "Cyto")
GActin = Species("GActin", 482.4, vol_unit, 13.37, D_unit, "Cyto")
LaminA = Species("LaminA", 0.0, surf_unit, 0.001, D_unit, "NM")
NPC_A = Species("NPC_A", 0.0, surf_unit, 0.001, D_unit, "NM")
YAPTAZ = Species("YAPTAZ", 0.7, vol_unit, 19.0, D_unit, "Cyto") #non-phosphorylated in the cytosol
YAPTAZ_phos = Species("YAPTAZ_phos", 0.2, vol_unit, 19.0, D_unit, "Cyto") #phosphorylated in the cytosol
YAPTAZ_nuc = Species("YAPTAZ_nuc", 0.7, vol_unit, 19.0, D_unit, "Nuc")
MRTF = Species("MRTF", 0.0, vol_unit, 5.0, D_unit, "Cyto") #free MRTF in the cytosol, very rough estimate for diffusion
MRTF_bound = Species("MRTF_bound", 0.65, vol_unit, 5.0, D_unit, "Cyto") #free MRTF in the cytosol, very rough estimate for diffusion
MRTF_nuc = Species("MRTF_nuc", 0.301, vol_unit, 5.0, D_unit, "Nuc")

In [6]:
E = 1.0 # substrate stiffness in kPa
# now define reactions and parameters across 3 modules
# module A: Substrate stiffness -> initial signaling events
# a1: FAK phosphorylation and dephosphorylation
FAK_tot = Parameter("FAK_tot", 1.0, vol_unit)
k_f = Parameter("k_f", 0.015, 1/sec)
k_sf = Parameter("k_sf", 0.379, 1/sec)
stiffnessFactor = Parameter("stiffnessFactor", E/(E + 3.25), dimensionless)
k_df = Parameter("k_df", 0.035, 1/sec)
a1 = Reaction("a1", [], ["pFAK"], 
              param_map={"FAK_tot":"FAK_tot", "k_f":"k_f", "k_sf":"k_sf", "stiffnessFactor":"stiffnessFactor", "k_df":"k_df"}, 
              species_map={"pFAK":"pFAK"}, 
              eqn_f_str="(FAK_tot-pFAK)*(k_f + k_sf*stiffnessFactor) - k_df*pFAK")

# a2: RhoA activation and deactivation
k_fkrho = Parameter("k_fkrho", 0.0168, 1/sec)
gammaConst = Parameter("gammaConst", 77.56, uM**(-5))
# n = Parameter("n", 5, dimensionless)
k_drho = Parameter("k_drho", 0.625, 1/sec)
vol_cyto = (4/3) * np.pi * (cell_rad**3 - nuc_rad**3)
vol_nuc = (4/3) * np.pi * nuc_rad**3
sa_pm = 4 * np.pi * cell_rad**2
cytoConvert = Parameter("cytoConvert", vol_cyto/sa_pm, um)
a2 = Reaction("a2", ["RhoA_GDP"], ["RhoA_GTP"], 
              param_map={"k_fkrho":"k_fkrho", "gammaConst":"gammaConst", "k_drho":"k_drho", 
                         "cytoConvert":"cytoConvert"}, 
              species_map={"pFAK":"pFAK", "RhoA_GDP":"RhoA_GDP", "RhoA_GTP":"RhoA_GTP"}, 
              eqn_f_str="RhoA_GDP*cytoConvert*k_fkrho*(gammaConst*pFAK**5 + 1)",
              eqn_r_str="k_drho*RhoA_GTP")

#a3: ROCK activation by RhoA_GTP at PM
k_rrho = Parameter("k_rrho", 0.648, 1/(sec*uM))
ROCK_tot = Parameter("ROCK_tot", 1.0, uM)
a3 = Reaction("a3", [], ["ROCK_A"], 
              param_map={"k_rrho":"k_rrho", "ROCK_tot":"ROCK_tot"}, 
              species_map={"ROCK_A":"ROCK_A", "RhoA_GTP":"RhoA_GTP"}, 
              eqn_f_str="k_rrho*RhoA_GTP*(ROCK_tot-ROCK_A)")

#a4: Deactivation of ROCK in the cytosol
k_drock = Parameter("k_drock", 0.8, 1/sec)
a4 = Reaction("a4", ["ROCK_A"], [], 
              param_map={"k_drock":"k_drock"}, 
              species_map={"ROCK_A":"ROCK_A"}, 
              eqn_f_str="k_drock*ROCK_A")

In [7]:
#Module B: cytoskeletal signaling
#b1: mDia activation by RhoA_GTP at PM
k_mrho = Parameter("k_mrho", 0.002, 1/(sec*uM))
mDia_tot = Parameter("mDia_tot", 0.8, uM)
b1 = Reaction("b1", [], ["mDia_A"], 
              param_map={"k_mrho":"k_mrho", "mDia_tot":"mDia_tot"}, 
              species_map={"mDia_A":"mDia_A", "RhoA_GTP":"RhoA_GTP"}, 
              eqn_f_str="k_mrho*RhoA_GTP*(mDia_tot-mDia_A)")

#b2: Deactivation of mDia in the cytosol
k_dmdia = Parameter("k_dmdia", 0.005, 1/sec)
b2 = Reaction("b2", ["mDia_A"], [], 
              param_map={"k_dmdia":"k_dmdia"}, 
              species_map={"mDia_A":"mDia_A"}, 
              eqn_f_str="k_dmdia*mDia_A")

#b3: activation and deactivation of myosin
Myo_tot = Parameter("Myo_tot", 5.0, vol_unit)
k_mr = Parameter("k_mr", 0.03, 1/sec)
ROCK_B = Parameter("ROCK_B", 0.3, vol_unit)
epsilon = Parameter("epsilon", 36.0, 1/uM)
sc1 = Parameter("sc1", 20.0, 1/uM)
k_dmy = Parameter("k_dmy", 0.067, 1/sec)
b3 = Reaction("b3", [], ["Myo_A"], 
              param_map={"Myo_tot":"Myo_tot", "k_mr":"k_mr", "ROCK_B":"ROCK_B", "epsilon":"epsilon", "sc1":"sc1", "k_dmy":"k_dmy"}, 
              species_map={"Myo_A":"Myo_A", "ROCK_A":"ROCK_A"}, 
              eqn_f_str="(Myo_tot-Myo_A)*k_mr*(1 + epsilon*(ROCK_A/2)*(tanh(sc1*(ROCK_A-ROCK_B)) + 1))\
                         - k_dmy*Myo_A")

#b4: activation and deactivation of LIMK
LIMK_tot = Parameter("LIMK_tot", 2.0, vol_unit)
k_lr = Parameter("k_lr", 0.07, 1/sec)
tau = Parameter("tau", 55.49, 1/uM)
k_dl = Parameter("k_dl", 2.0, 1/sec)
b4 = Reaction("b4", [], ["LIMK_A"], 
              param_map={"LIMK_tot":"LIMK_tot", "k_lr":"k_lr", "ROCK_B":"ROCK_B", "tau":"tau", "sc1":"sc1", "k_dl":"k_dl"}, 
              species_map={"LIMK_A":"LIMK_A", "ROCK_A":"ROCK_A"}, 
              eqn_f_str="(LIMK_tot-LIMK_A)*k_lr*(1 + tau*(ROCK_A/2)*(tanh(sc1*(ROCK_A-ROCK_B)) + 1))\
                         - k_dl*LIMK_A")

#b5: dephos. and phos. of Cofilin
Cofilin_tot = Parameter("Cofilin_tot", 2.0, vol_unit)
k_turnover = Parameter("k_turnover", 0.04, 1/sec)
k_catCof = Parameter("k_catCof", 0.34, 1/sec)
k_mCof = Parameter("k_mCof", 4.0, uM)
b5 = Reaction("b5", [], ["Cofilin_NP"], 
              param_map={"Cofilin_tot":"Cofilin_tot", "k_turnover":"k_turnover", "k_catCof":"k_catCof", "k_mCof":"k_mCof"}, 
              species_map={"LIMK_A":"LIMK_A", "Cofilin_NP":"Cofilin_NP"}, 
              eqn_f_str="(Cofilin_tot-Cofilin_NP)*k_turnover - k_catCof*LIMK_A*Cofilin_NP/(k_mCof + Cofilin_NP)")

#b6: actin polymerization and depolymerization
# Actin_tot = Parameter("Actin_tot", 500.0, uM)
k_ra = Parameter("k_ra", 0.4, 1/sec)
alpha = Parameter("alpha", 50.0, 1/uM)
mDia_B = Parameter("mDia_B", 0.165, uM)
k_dep = Parameter("k_dep", 3.5, 1/sec)
k_fc1 = Parameter("k_fc1", 4.0, 1/(uM*sec))
b6 = Reaction("b6", ["GActin"], ["FActin"], 
              param_map={"k_ra":"k_ra", "alpha":"alpha", "sc1":"sc1", "mDia_B":"mDia_B", "k_dep":"k_dep", "k_fc1":"k_fc1"}, 
              species_map={"FActin":"FActin", "GActin":"GActin", "mDia_A":"mDia_A", "Cofilin_NP":"Cofilin_NP"}, 
              eqn_f_str="GActin*k_ra*(1 + alpha*(mDia_A/2)*(tanh(sc1*(mDia_A-mDia_B)) + 1))\
                         - (k_dep + k_fc1*Cofilin_NP)*FActin")

In [8]:
#Module C: nucleo-cytoplasmic transport
#c1: YAP/TAZ dephos. and phos. in the cytosol
# YAPTAZ_tot = Parameter("YAPTAZ_tot", 1385000, molecule)
k_CN = Parameter("k_CN", 0.56, 1/sec)
k_CY = Parameter("k_CY", 0.00076, 1/(sec*uM**2))
k_NC = Parameter("k_NC", 0.14, 1/sec)
# concToNum_cyto = Parameter("concToNum_cyto", 602.214*vol_cyto, molecule/uM)
# concToNum_nuc = Parameter("concToNum_nuc", 602.214*vol_nuc, molecule/uM)
c1 = Reaction("c1", ["YAPTAZ_phos"], ["YAPTAZ"], 
              param_map={"k_CN":"k_CN", "k_CY":"k_CY", "k_NC":"k_NC"}, 
              species_map={"YAPTAZ":"YAPTAZ", "YAPTAZ_phos":"YAPTAZ_phos", "FActin":"FActin", "Myo_A":"Myo_A"}, 
              eqn_f_str="YAPTAZ_phos*(k_CN + k_CY*FActin*Myo_A) - k_NC*YAPTAZ")

#c2: MRTF binding/unbinding from G-actin in the cytosol
# MRTF_tot = Parameter("YAPTAZ_tot", 1000000, molecule)
k_offMRTF = Parameter("k_offMRTF", 1.0, 1/sec)
k_onMRTF = Parameter("k_onMRTF", 2.3e-5, 1/(sec*uM**2))
c2 = Reaction("c2", ["MRTF_bound"], ["MRTF"], 
              param_map={"k_offMRTF":"k_offMRTF", "k_onMRTF":"k_onMRTF"}, 
              species_map={"MRTF_bound":"MRTF_bound", "GActin":"GActin", "MRTF":"MRTF"}, 
              eqn_f_str="MRTF_bound*k_offMRTF - k_onMRTF*GActin**2 * MRTF")

#c3: LaminA dephos. and phos.
LaminA_tot = Parameter("LaminA_tot", 3500.0, surf_unit)
k_fl = Parameter("k_fl", 0.46, 1/sec)
p = Parameter("p", 9.0e-6, kPa/uM**2.6)
C_LaminA = Parameter("C_LaminA", 100.0, kPa)
k_rl = Parameter("k_rl", 0.001, 1/sec)
c3 = Reaction("c3", [], ["LaminA"], 
              param_map={"LaminA_tot":"LaminA_tot", "k_fl":"k_fl", "p":"p", "C_LaminA":"C_LaminA", "k_rl":"k_rl"}, 
              species_map={"LaminA":"LaminA", "FActin":"FActin"}, 
              eqn_f_str="(LaminA_tot - LaminA)*k_fl*p*FActin**2.6/(C_LaminA + p*FActin**2.6) - k_rl*LaminA")

#c4: opening and closing of NPCs
NPC_tot = Parameter("NPC_tot", 6.5, surf_unit)
k_fNPC = Parameter("k_fNPC", 2.8e-7, 1/(sec*uM**2 *surf_unit))
k_rNPC = Parameter("k_rNPC", 8.7, 1/sec)
c4 = Reaction("c4", [], ["NPC_A"], 
              param_map={"NPC_tot":"NPC_tot", "k_fNPC":"k_fNPC", "k_rNPC":"k_rNPC"}, 
              species_map={"NPC_A":"NPC_A", "LaminA":"LaminA", "FActin":"FActin", "Myo_A":"Myo_A"}, 
              eqn_f_str="(NPC_tot - NPC_A)*k_fNPC*LaminA*FActin*Myo_A - k_rNPC*NPC_A")

#c5: nuclear translocation of YAP/TAZ
k_insolo = Parameter("k_insolo", 1.0, surf_unit/(sec*uM))
k_in2 = Parameter("k_in2", 10.0, 1/(sec*uM))
k_out = Parameter("k_out", 1.0, surf_unit/(sec*uM))
c5 = Reaction("c5", ["YAPTAZ"], ["YAPTAZ_nuc"], 
              param_map={"k_insolo":"k_insolo", "k_in2":"k_in2", "k_out":"k_out"}, 
              species_map={"YAPTAZ":"YAPTAZ", "YAPTAZ_nuc":"YAPTAZ_nuc", "NPC_A":"NPC_A"}, 
              eqn_f_str="YAPTAZ*(k_insolo + k_in2*NPC_A) - k_out*YAPTAZ_nuc")

#c6: nuclear translocation of MRTF
k_insoloMRTF = Parameter("k_insoloMRTF", 2.708, surf_unit/(sec*uM))
k_in2MRTF = Parameter("k_in2MRTF", 8.5, 1/(sec*uM))
k_outMRTF = Parameter("k_outMRTF", 1.0, surf_unit/(sec*uM))
c6 = Reaction("c6", ["MRTF"], ["MRTF_nuc"], 
              param_map={"k_insoloMRTF":"k_insoloMRTF", "k_in2MRTF":"k_in2MRTF", "k_outMRTF":"k_outMRTF"}, 
              species_map={"MRTF":"MRTF", "MRTF_nuc":"MRTF_nuc", "NPC_A":"NPC_A"}, 
              eqn_f_str="MRTF*(k_insoloMRTF + k_in2MRTF*NPC_A) - k_outMRTF*MRTF_nuc")

In [9]:

pc, sc, cc, rc = sbmodel_from_locals(locals().values())

In [10]:
configCur = config.Config()
configCur.flags.update({"allow_unused_components": True})
model_cur = model.Model(pc, sc, cc, rc, configCur, parent_mesh)
configCur.solver.update(
    {
        "final_t": 1000.0,
        "initial_dt": 10.0,
        "time_precision": 8,
        "use_snes": True
    }
)

model_cur.initialize(initialize_solver=True)




 2023-10-23 18:51:04,972 smart.model - DEBUG - Checking validity of model (step 1 of ZZ) (model.py:239) 


2023-10-23 18:51:04,973 smart.model - DEBUG - Check that mesh/compartment dimensionalities match (model.py:324) 
2023-10-23 18:51:04,974 smart.model - DEBUG - Checking for namespace conflicts (model.py:360) 
 2023-10-23 18:51:04,974 smart.model - DEBUG - Step 1 of initialization completed successfully! (model.py:243) 



 2023-10-23 18:51:04,975 smart.model - DEBUG - Cross-Container Dependent Initializations (step 2 of ZZ) (model.py:251) 


2023-10-23 18:51:04,975 smart.model - DEBUG - Turning reactions into unsigned symbolic flux strings (model.py:413) 
2023-10-23 18:51:05,077 smart.model - DEBUG - Make sure all reactions have parameters/species defined (model.py:457) 
2023-10-23 18:51:05,093 smart.model - DEBUG - Linking parameters, species, and compartments to reactions (model.py:479) 
2023-10-23 18:51:05,094 smart.model - DEBUG - Checking for unused parameters, species, or 

2023-10-23 18:51:05,630 smart.model_assembly - DEBUG - Getting fluxes for reaction b6 (model_assembly.py:1249) 
2023-10-23 18:51:05,713 smart.model_assembly - DEBUG - Getting fluxes for reaction b5 (model_assembly.py:1249) 
2023-10-23 18:51:05,732 smart.model_assembly - DEBUG - Getting fluxes for reaction b4 (model_assembly.py:1249) 
2023-10-23 18:51:05,800 smart.model_assembly - DEBUG - Getting fluxes for reaction b3 (model_assembly.py:1249) 
2023-10-23 18:51:05,818 smart.model_assembly - DEBUG - Getting fluxes for reaction b2 (model_assembly.py:1249) 
2023-10-23 18:51:05,823 smart.model_assembly - DEBUG - Getting fluxes for reaction b1 (model_assembly.py:1249) 
2023-10-23 18:51:05,827 smart.model_assembly - DEBUG - Flux b1 [mDia_A (f)] scaled by 0.0016605390404271648 micrometer ** 3 * micromolar / molecule (model_assembly.py:1498) 
2023-10-23 18:51:05,827 smart.model_assembly - DEBUG - Old flux units: 1 micrometer * micromolar / second (model_assembly.py:1502) 
2023-10-23 18:51:05,82

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-10-23 18:51:07,320 smart.model - INFO - J01 = dF[NM])/du[Nuc] is empty (model.py:1397) 
2023-10-23 18:51:07,320 smart.model - INFO - J02 = dF[NM])/du[PM] is empty (model.py:1397) 
2023-10-23 18:51:07,325 smart.model - INFO - J12 = dF[Nuc])/du[PM] is empty (model.py:1397) 
2023-10-23 18:51:07,327 smart.model - INFO - J20 = dF[PM])/du[NM] is empty (model.py:1397) 
2023-10-23 18:51:07,327 smart.model - INFO - J21 = dF[PM])/du[Nuc] is empty (model.py:1397) 
ROOT -2023-10-23 18:51:07,431 FFC - WARNING -   WARNING: The number of integration points for each cell will be: 729 (log.py:153)
ROOT -2023-10-23 18:51:07,432 FFC - WARNING -            Consider using the option 'quadrature_degree' to reduce the number of points (log.py:153)


In [ ]:
# Write initial condition(s) to file
results = dict()
result_folder = pathlib.Path(f"results_sphere_10sTimestep")
result_folder.mkdir(exist_ok=True)
for species_name, species in model_cur.sc.items:
    results[species_name] = d.XDMFFile(
        model_cur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
model_cur.to_pickle("model_cur.pkl")

# Set loglevel to warning in order not to pollute notebook output
# logger.setLevel(logging.WARNING)
# Solve
displayed = False
while True:
    # Solve the system
    model_cur.monolithic_solve()
    # Save results for post processing
    for species_name, species in model_cur.sc.items:
        results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
    np.savetxt(result_folder / f"tvec.txt", np.array(model_cur.tvec).astype(np.float32))
    # End if we've passed the final time
    if model_cur.t >= model_cur.final_t:
        break